In [10]:
import torch
from torch import nn

import numpy as np
import random
import math


In [11]:
with open('data/train-consonant-examples.txt') as f:
    consonant_train = f.read().splitlines()
    
with open('data/train-vowel-examples.txt') as f:
    vowel_train = f.read().splitlines()

with open('data/dev-consonant-examples.txt') as f:
    consonant_test = f.read().splitlines()
    
with open('data/dev-vowel-examples.txt') as f:
    vowel_test = f.read().splitlines()
    
    
chars = set(''.join(consonant_train))
ichars = set(''.join(vowel_train))

char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))


vowels = {0:'a',1:'e',2:'i', 3:'o',4:'u'}
consonants = {0:'b',1:'c',2:'d',3:'f',4:'g',5:'h',5:'j',6:'k',7:'l',8:'m',9:'n',10:'p',11:'q',12:'r',13:'s',14:'t',15:'v',16:'w',17:'x',18:'y',19:'z'}


    

cons_target = []
vow_target = []

test_cons_target = []
test_vow_target = []


for i in range(len(consonant_train)):

    cons_target.append(0)
    
for i in range(len(vowel_train)):
    
    vow_target.append(1)
    
for i in range(len(consonant_test)):

    test_cons_target.append(0)
    
for i in range(len(vowel_test)):
    
    test_vow_target.append(1)
    

cons_input_seq = []
vow_input_seq = []

cons_target_seq = []
vow_target_seq = []


test_cons_input_seq = []
test_vow_input_seq = []

test_cons_target_seq = []
test_vow_target_seq = []



for i in consonant_test:
    test_cons_input_seq.append([char_to_int[char] for char in i])
    
for j in vowel_test:
    test_vow_input_seq.append([char_to_int[char] for char in j])
    
for i in consonant_train:
    cons_input_seq.append([char_to_int[char] for char in i])
    
for j in vowel_train:
    vow_input_seq.append([char_to_int[char] for char in j])
    


   

    


input_seq = cons_input_seq + vow_input_seq
target_seq = cons_target + vow_target


tar_seq = np.asarray(target_seq)



test_input_seq = test_cons_input_seq + test_vow_input_seq
test_target_seq = test_cons_target + test_vow_target


test_tar_seq = np.asarray(test_target_seq)

In [12]:
print(char_to_int)

{'a': 0, 'x': 1, 'g': 2, 'w': 3, 'y': 4, 'q': 5, 'v': 6, 't': 7, 'z': 8, 'd': 9, ' ': 10, 'f': 11, 'r': 12, 'p': 13, 's': 14, 'o': 15, 'n': 16, 'c': 17, 'i': 18, 'k': 19, 'l': 20, 'm': 21, 'u': 22, 'e': 23, 'h': 24, 'j': 25, 'b': 26}


In [13]:
inp_seq = np.asarray(input_seq)
inp_seq = torch.tensor(inp_seq)
tar_seq = torch.tensor(target_seq)

test_inp_seq = np.asarray(test_input_seq)
test_inp_seq = torch.tensor(test_inp_seq)
test_tar_seq = torch.tensor(test_target_seq)

In [14]:
print(inp_seq.shape)
print(tar_seq.shape)

print(test_inp_seq.shape)
print(test_tar_seq.shape)

torch.Size([10000, 20])
torch.Size([10000])
torch.Size([1000, 20])
torch.Size([1000])


In [15]:
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [16]:
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(inp_seq, tar_seq)
test_data = TensorDataset(inp_seq, tar_seq)

batch_size = 100

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [17]:
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print(sample_x.shape, sample_y.shape)

torch.Size([100, 20]) torch.Size([100])


In [38]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers):
        super(Model, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        print(x.shape)
        print("batch_size: ", batch_size)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        

        out = self.fc(lstm_out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [39]:
vocab_size = len(char_to_int) + 1
output_size = 1
embedding_dim = 100
hidden_dim = 128
n_layers = 2

model = Model(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.to(device)
print(model)

Model(
  (embedding): Embedding(28, 100)
  (lstm): LSTM(100, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [40]:
lr=0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [41]:
epochs = 10
counter = 0
print_every = 1000


model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1

        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

    print("Epoch: {}/{}...".format(i+1, epochs),
          "Step: {}...".format(counter),
           "Loss: {:.6f}...".format(loss.item()))


torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
bat

torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
bat

torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
bat

torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
bat

torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
batch_size:  100
torch.Size([100, 20])
bat

In [44]:
test_losses = []
num_correct = 0
h = model.init_hidden(batch_size)
print("len  ", h)

model.eval()
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, h = model(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    pred = torch.round(output.squeeze())
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
        
print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

len   (tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0'), tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0